In [129]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import seaborn as sns
import os
import math
import cv2
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [130]:
data = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_data = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
train_dir = '../input/petfinder-pawpularity-score/train/'

# Чистка даних

In [131]:
nan_values = data.isna()
nan_columns = nan_values.any()

In [132]:
nan_columns

In [133]:
data.isnull().sum()

In [134]:
data.Id.nunique() == len(data)

Нема null значень, айдішки не дублікуються. Чистка не потрібна

# Підрахунок

In [135]:
data.describe()

In [136]:
data.Pawpularity.describe()

In [137]:
data.iloc[:, 1:].std()

# Візуалізація та кореляція

In [138]:
data.Pawpularity.hist(bins=50)

In [139]:
import matplotlib.pyplot as plt 
f, ax = plt.subplots(2, 6, figsize=(16, 12))
ax = ax.flatten()
for idx, col in enumerate(list(data.columns)[1:-1]):
    sns.histplot(data[[col, "Pawpularity"]], hue=col, x="Pawpularity", kde=True, ax=ax[idx])
    ax[idx].axvline(data['Pawpularity'].mean(),c='purple',ls='-',lw=3)
    

In [140]:
plt.figure(figsize=(14,7))
sns.heatmap(data.iloc[:, 1:].corr(), annot = True)

# Аномалії

In [141]:
data_Pawpularity = np.std(data.Pawpularity)
data_mean = np.mean(data.Pawpularity)
anomally_cut_off = data_std * 3 # upper and lower limit to 3 standart derivations
lower_limit = data_mean - anomally_cut_off
upper_limit = data_mean+anomally_cut_off

In [142]:
lower_limit

In [143]:
upper_limit

In [144]:
anomalies = data[(data.Pawpularity > upper_limit) | (data.Pawpularity < lower_limit)]
        


In [145]:
anomalies

In [146]:
def see_animals(df):
    
    max_imgs = min(18,len(df))
    
    plt.figure(figsize=(15, 30))
    cols = 3
    rows = int(math.ceil(max_imgs / cols))
    
    for i in range(max_imgs):
        
        row = df.iloc[i]
        im_path = os.path.join(train_dir,row['Id']+".jpg")
        im = cv2.imread(im_path)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        plt.subplot(rows,cols,i+1)       
        Pawpularity = row['Pawpularity']
        title = f"Pawpularity: {Pawpularity}, "
        
        for i, col in enumerate(data.columns[1:-1]):
            if i % 3 == 0 :
                title+="\n"
            title+=f"{col}: {row[col]}, "
            
                
        plt.title(title)
        
        plt.imshow(im)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    plt.close()

In [147]:
see_animals(anomalies)

In [148]:
anomalies[anomalies.Pawpularity > 99].Id.count()

In [149]:
anomalies[anomalies.Pawpularity < 100].Id.count()

**Більшість аномалій з високим Pawpularity скором. По картинкам аномалій не видно (реально милі тваринки))))**

# Порівняння test та train

In [150]:
data.shape

In [151]:
test_data.shape

In [157]:
test_data.isnull().sum()

# Pandas Profile Report

In [152]:
from pandas_profiling import ProfileReport

In [153]:
profile = ProfileReport(data, explorative=True)

In [154]:
profile